In [1]:
# !pip install --upgrade pip
! pip install haystack 
!pip install haystack-ai
# !pip install farm-haystack[colab,inference]
# !pip3.10 uninstall pydantic urllib3
!pip install farm-haystack[colab,preprocessing,elasticsearch,inference]
!pip install -r requirements.txt 

  Using cached haystack-0.42-py2.py3-none-any.whl.metadata (10 kB)
  Using cached construct-2.5.3-py2.py3-none-any.whl
  Using cached pefile-2023.2.7-py3-none-any.whl.metadata (1.4 kB)
  Using cached python_ptrace-0.9.9-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached haystack-0.42-py2.py3-none-any.whl (179 kB)
Using cached python_ptrace-0.9.9-py2.py3-none-any.whl (104 kB)
Using cached pefile-2023.2.7-py3-none-any.whl (71 kB)
  Using cached farm_haystack-1.26.2-py3-none-any.whl.metadata (31 kB)
  Using cached boilerpy3-1.0.7-py3-none-any.whl.metadata (5.8 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached prompthub_py-4.0.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pydantic-1.10.17-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (151 kB)
  Using cached quantulum3-0.9.2-py3-none-any.whl.metadata (16 kB)
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (

In [2]:
from haystack import Pipeline
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack import Document

# Function to read text file
def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found. Please make sure the file exists and the path is correct.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {str(e)}")
        return None

# Read the text file
file_path = "1.PMAY-U 2.0 FAQ.txt"  # Assuming you've converted the .docx to .txt
document_content = read_text_file(file_path)

if document_content is None:
    print("Exiting due to file read error.")
    exit(1)

# Initialize QdrantDocumentStore
document_store = QdrantDocumentStore(
    url='http://localhost:6333',
    recreate_index=True,
    return_embedding=True,
    wait_result_from_api=True,
)

# Create a Document object
document = Document(content=document_content)

# Initialize and use SentenceTransformersDocumentEmbedder
document_embedder = SentenceTransformersDocumentEmbedder()
document_embedder.warm_up()
documents_with_embeddings = document_embedder.run([document])

# Write document to the document store
document_store.write_documents(documents_with_embeddings.get("documents"), policy=DuplicatePolicy.OVERWRITE)

# Initialize QdrantEmbeddingRetriever
retriever = QdrantEmbeddingRetriever(document_store=document_store)

# Define the prompt template
template = """
Given only the following information from the PMAY-U 2.0 FAQ document, answer the question.
Ignore your own knowledge.
Context:
{% for document in documents %}
 {{ document.content }}
{% endfor %}
Question: {{ query }}
"""

# Set up the pipeline
pipe = Pipeline()
pipe.add_component("text_embedder", SentenceTransformersTextEmbedder())
pipe.add_component("retriever", retriever)
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", OllamaGenerator(model="llama3", url="http://localhost:6333/api/generate"))
pipe.connect("text_embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class 'pandas.core.frame.DataFrame'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.8/u/schema-for-unknown-type

In [ ]:
# Example query
query = "How Central Assistance will be released for Private Sector AHP Projects ?"
response = pipe.run({"prompt_builder": {"query": query}, "text_embedder": {"text": query}})
print(response["llm"]["replies"])

Batches: 100%|██████████| 1/1 [00:00<00:00, 23.92it/s]


HTTPError: 404 Client Error: Not Found for url: http://localhost:6333/api/generate